In [1]:
import pickle
import math

from nltk import word_tokenize
from nltk.translate.bleu_score import modified_precision, closest_ref_length, brevity_penalty, SmoothingFunction, sentence_bleu

from collections import Counter

from fractions import Fraction

from modules.sentence import tokenizer, read, detokenize
from modules.model import NMT

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
nbest = 0
weights = (0.25, 0.25, 0.25, 0.25)

In [3]:
from models import fl_en
model = fl_en

In [14]:
source_tokenizer = tokenizer(model.config['source_tokenizer'], lowercase=model.config['source_lowercase'])
source_eval = read('models/fl_en/source.data.test', source_tokenizer, model.config['backwards'])

In [ ]:
target_tokenizer = tokenizer('word', lowercase=model.config['target_lowercase'])
references = read('models/fl_en/source.data.test', target_tokenizer, model.config['backwards'])
print(references)

In [ ]:
output_file = open('models/fl_en/result.data.test', 'w', encoding='utf-8')

hypotheses = []

for i, sent in enumerate(model.translate(source_eval, encode=True, nbest=nbest)):
    print(sent, file=output_file, flush=True)
    hypotheses.append(word_tokenize(sent))
    
output_file.close()

In [1]:
evaluation_file = open('models/fl_en/scores.data.eval.csv','w', encoding='utf-8')

p_numerators = Counter()
p_denominators = Counter()
hyp_lengths, ref_lengths = 0, 0

for reference, hypothesis in zip(references, hypotheses):
    
    hyp_len = len(hypothesis)
    ref_len = closest_ref_length([reference], hyp_len)
    
    hyp_lengths += hyp_len
    ref_lengths += ref_len
    
    set_data = '%d,%d' % (ref_len, hyp_len)
    
    for i, _ in enumerate(weights, start=1):
        p_i = modified_precision(reference, hypothesis, i)
        p_numerators[i] += p_i.numerator
        p_denominators[i] += p_i.denominator
        set_data += ',%d,%d' % (p_i.numerator, p_i.denominator)
    
    set_data += ',%f' % sentence_bleu([reference], hypothesis)
    
    print(set_data, file=evaluation_file, flush=True)
        
evaluation_file.close()

NameError: name 'Counter' is not defined

In [51]:
bp = brevity_penalty(ref_lengths, hyp_lengths)
print(bp)

0.920491284044873


In [52]:
p_n = [Fraction(p_numerators[i], p_denominators[i], _normalize=False)
    for i, _ in enumerate(weights, start=1)]

In [53]:
smoothing_function = SmoothingFunction().method0

p_n = smoothing_function(p_n, references=references, hypothesis=hypothesis,
                             hyp_len=hyp_len, emulate_multibleu=False)

s = (w * math.log(p_i) for i, (w, p_i) in enumerate(zip(weights, p_n)))

print(math.exp(math.fsum(s)))

0.01839351702653491


In [54]:
from nltk.translate.bleu_score import corpus_bleu

corpus_bleu(references, hypotheses)

/mnt/c/ubuntu-bash/filipineu/venv/lib/python3.5/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


0.051233175968410725